# Step 1
<h2 style="color:red"> Transposing the datasets </h2>

As described in the report, we operate a specific transposition to obtain a better shape of the datasets.

In [ ]:
import pandas as pd

X = pd.read_csv("InputTrain.csv").iloc[:,2:]
dfXTest = pd.read_csv("InputTest.csv").iloc[:,2:]
Y1 = pd.read_csv("StepTwo_LabelTrain_WashingMachine.csv").iloc[:,2:]
Y2 = pd.read_csv("StepTwo_LabelTrain_Dishwasher.csv").iloc[:,2:]
Y3 = pd.read_csv("StepTwo_LabelTrain_TumbleDryer.csv").iloc[:,2:]
Y4 = pd.read_csv("StepTwo_LabelTrain_Microwave.csv").iloc[:,2:]
Y5 = pd.read_csv("StepTwo_LabelTrain_Kettle.csv").iloc[:,2:]

In [ ]:
from sklearn.preprocessing import StandardScaler
consumption_values = dfXTest.values
consumption_values = consumption_values.reshape(consumption_values.shape[0], -1)

# Remove the mean and scale the values to have unit variance
scaler = StandardScaler()
normalized_consumption_values = scaler.fit_transform(consumption_values)
dfXTest.iloc[:, 0:] = normalized_consumption_values.reshape(normalized_consumption_values.shape[0], -1)

from sklearn.preprocessing import StandardScaler
consumption_values = X.values
consumption_values = consumption_values.reshape(consumption_values.shape[0], -1)

# Remove the mean and scale the values to have unit variance
scaler = StandardScaler()
normalized_consumption_values = scaler.fit_transform(consumption_values)
X.iloc[:, 0:] = normalized_consumption_values.reshape(normalized_consumption_values.shape[0], -1)

In [ ]:
y1 = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(Y1)):
    y1 = pd.concat([y1, pd.DataFrame(Y1.iloc[i].values.reshape(1, -1).T)])

y2 = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(Y1)):
    y2 = pd.concat([y2, pd.DataFrame(Y2.iloc[i].values.reshape(1, -1).T)])

y3 = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(Y3)):
    y3 = pd.concat([y3, pd.DataFrame(Y3.iloc[i].values.reshape(1, -1).T)])


y4 = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(Y4)):
    y4 = pd.concat([y4, pd.DataFrame(Y4.iloc[i].values.reshape(1, -1).T)])

y5 = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(Y5)):
    y5 = pd.concat([y5, pd.DataFrame(Y5.iloc[i].values.reshape(1, -1).T)])

x = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(X)):
    x = pd.concat([x, pd.DataFrame(X.iloc[i].values.reshape(1, -1).T)])



dfTest = pd.DataFrame()

# Iterate over each row of the original dataframe and append the transposed row to the new dataframe
for i in range(len(dfXTest)):
    dfTest = pd.concat([dfTest, pd.DataFrame(dfXTest.iloc[i].values.reshape(1, -1).T)])

x.to_csv("x_transposed_notNormalized.csv")
dfTest.to_csv("dfTest_transposed_notNormalized.csv")

# Step 2
<h2 style="color:red"> Adding the features  </h2>

In the following part, we will add the features

In [ ]:
import pandas as pd
x = pd.read_csv("../x_transposed_notNormalized.csv")
dfxTest= pd.read_csv("../dfTest_transposed_notNormalized.csv")

listI = [10,20,50,60,65,70,80,90,100,120,110]
for i in listI:
    x[f'consumption_lag_{i}'] = x['0'].shift(i)
    dfxTest[f'consumption_lag_{i}'] = dfxTest['0'].shift(i)


# replace NaN values with previous non-NaN values
x = x.fillna(method='bfill')
dfxTest = dfxTest.fillna(method='bfill')
x = x.drop('Unnamed: 0',axis = 1)

dfxTest = dfxTest.drop('Unnamed: 0',axis = 1)


In [ ]:
x['rolling_skewness20'] = x['0'].rolling(20).skew()
dfxTest['rolling_skewness20'] = dfxTest['0'].rolling(20).skew()
x['rolling_kurtosis20'] = x['0'].rolling(20).kurt()
dfxTest['rolling_kurtosis20'] = dfxTest['0'].rolling(20).kurt() 

In [ ]:
x['rolling_std2'] = x['rolling_std2'].interpolate(method='linear', limit_direction='both')
x['rolling_std3'] = x['rolling_std3'].interpolate(method='linear', limit_direction='both')
dfxTest['rolling_std2'] = dfxTest['rolling_std2'].interpolate(method='linear', limit_direction='both')
dfxTest['rolling_std3'] = dfxTest['rolling_std3'].interpolate(method='linear', limit_direction='both')

x['rolling_skewness20'] = x['rolling_skewness20'].interpolate(method='linear', limit_direction='both')

x['rolling_kurtosis20'] = x['rolling_kurtosis20'].interpolate(method='linear', limit_direction='both')

dfxTest['rolling_skewness20'] = dfxTest['rolling_skewness20'].interpolate(method='linear', limit_direction='both')

dfxTest['rolling_kurtosis20'] = dfxTest['rolling_kurtosis20'].interpolate(method='linear', limit_direction='both')

In [ ]:
import numpy as np
from scipy.signal import argrelmax, argrelmin

def time_since_last_extreme(data, find_extreme_func):
    extreme_indices, = find_extreme_func(data)  # Unpack the tuple
    time_since_last_extreme = np.zeros_like(data)
    for idx in range(1, len(extreme_indices)):
        time_since_last_extreme[extreme_indices[idx-1]:extreme_indices[idx]] = np.arange(extreme_indices[idx] - extreme_indices[idx-1])
    time_since_last_extreme[extreme_indices[-1]:] = np.arange(len(data) - extreme_indices[-1])
    return time_since_last_extreme

x['time_since_last_max'] = time_since_last_extreme(x['0'].values, argrelmax)
x['time_since_last_min'] = time_since_last_extreme(x['0'].values, argrelmin)
dfxTest['time_since_last_max'] = time_since_last_extreme(dfxTest['0'].values, argrelmax)
dfxTest['time_since_last_min'] = time_since_last_extreme(dfxTest['0'].values, argrelmin)

In [ ]:
x.to_csv('../x-official_df.csv')
dfxTest.to_csv('../test-official_df.csv')

# Step 3
<h2 style="color:red"> Predictions  </h2>

In the final part, we will train,test the model, and then run the model on the final dataset to export to Kaggle

In [ ]:
import pandas as pd

x = pd.read_csv("../x-official_df.csv")
x = x.drop("Unnamed: 0",axis=1)
y = pd.read_csv("../y_ready2.csv")
y = y.drop("Unnamed: 0",axis=1)


In [ ]:
import gc
gc.collect()

In [ ]:
import xgboost as xgb
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3,random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3,shuffle=False)


dtrain = xgb.DMatrix(X_train,label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)

model = xgb.XGBModel(n_estimators=150)
params = {
    'max_depth': 10,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'map',
    'booster':'gbtree',
    'normalize_type':'forest',
    'learning_rate':0.4,
    'tree_method':'gpu_hist',
    'n_jobs':500,
    'colsample_bytree':0.8,
    'gamma':0.4,
    'subsample':0.8
}

# Train model
evalist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain,num_boost_round=60,verbose_eval=1,evals=evalist)

preds = model.predict(dtest) #Predictions
y_test = y_test.astype(float)
optimal_num_boost_round = model.best_iteration

# print("Precision = {}".format(average_precision_score(y_test.values, preds, average='macro')))




In [ ]:
dfxTest= pd.read_csv("../test-FInalMoreLag.csv")

dfxTest = dfxTest.drop('Unnamed: 0',axis=1)

dFin = xgb.DMatrix(dfxTest)

preds_f = model.predict(dFin)

dfF = pd.DataFrame(preds_f, columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
dfF = dfF.rename_axis('Index')
dfF = dfF.round(4)


In [ ]:
dfF.to_csv("../Final.csv")
